Content under Creative Commons Attribution license CC-BY 4.0, code under MIT license (c)2014 D. M. Phelps

# Backward Facing Step and Non Symmetrical Domains

## Background

So far in our lectures we have only discussed symmetical domains. That is to say we have dealt with boundries which are neat and orderly. In this lecture we will discuss how to incorperate steps and cavities into our domains. 

Many engineering problems studied today involve uniquely defined boundries. In this notebook we will look at the Backward Facing Step. The Backward Facing Step problem is a commonly studied problem which creates seperated flow and turbulance. It is defined as having free stream flow passing over a sharp ridge the flow will seperate at the ridge and reattach down stream.


## Equations

The governing equations for this flow will be Navier Stokes in two dimension and  an equation to relate pressure to velocity.


N.S. X direction
$$
\frac{\partial u}{\partial t} 
+u\frac{\partial u}{\partial x}
+v\frac{\partial u}{\partial y}
=
-\frac{1}{\rho}\frac{\partial p}{\partial x}
+
\nu\left(
\frac{\partial^2 u}{\partial x^2}
+
\frac{\partial^2 u}{\partial y^2}\right)
$$

N.S. Y direction
$$
\frac{\partial v}{\partial t} 
+v\frac{\partial v}{\partial x}
+u\frac{\partial v}{\partial y}
=
-\frac{1}{\rho}\frac{\partial p}{\partial y}
+
\nu\left(
\frac{\partial^2 v}{\partial y^2}
+
\frac{\partial^2 v}{\partial x^2}\right)
$$

Pressure
$$
\frac{\partial^2 p}{\partial x^2}
+
\frac{\partial^2 p}{\partial y^2}
=
-
\rho\left(
\frac{\partial u}{\partial x}
\frac{\partial u}{\partial x}
+
2
\frac{\partial u}{\partial x}
\frac{\partial v}{\partial y}
+
\frac{\partial v}{\partial y}
\frac{\partial v}{\partial y}
\right)
$$

Next will will discritize these equations
 <font color='red'>not done add discritizations
 </font>

## Boundry Conditions

discussion of boundry conditions

## Numerical Method

Discussion of the numerical method used

## Solving the problem

make and use the solver, plot the solved velocity field

## Convergence Check

check the convergence of our solution

## References

list of references used to create this notebook

http://lorenabarba.com/blog/cfd-python-12-steps-to-navier-stokes/